In [122]:
import pandas as pd
import numpy as np
import copy
#import unicode
import csv

In [140]:
class weatherData():
    
    def __init__(self,filename):
        self.xls = pd.ExcelFile(filename)
        self.df = pd.read_excel(self.xls, None)
        
    def getAggregate(self):
        df = self.df
        size = df['2009'].values.shape
        aggrData = np.empty((size), dtype = object)
        for year in df.keys():
            df[year]['Year'] = year
            #if(year == '2014'):
            #    print(df[year])
            yearWiseDataset = df[year].values
            if(aggrData.all() == None):
                aggrData = copy.deepcopy(yearWiseDataset)
                continue
            aggrData = np.concatenate((aggrData,yearWiseDataset), axis = 0)
        return aggrData
    
    def formatCol(self, col):
        col = [ 'Precip (mm) sum'  if 'Precip' in c else c for c in col ]
        return col
    '''
    def getWeatherEventData(self, weatherEvent):
        fog = []
        rain = []
        snow = []
        event = []
        for weat in weatherEvent:
            weat = str(weat)
            if weat == None or weat == 'NaN' or weat == 'nan':
                fog.append(0)
                rain.append(0)
                snow.append(0)
            else:
                weat = weat.lower()
                fog.append(1 if('fog' in weat)  else 0)
                rain.append(1 if('rain' in weat) else 0 )
                snow.append(1 if('snow' in weat) else 0)
        return fog,rain,snow
    '''
    def getWeatherEventData(self, weatherEvent):
        fog = []
        rain = []
        snow = []
        event = []
        for weat in weatherEvent:
            weat = str(weat)
            if weat == None or weat == 'NaN' or weat == 'nan':
                event.append(0)
            else:
                weat = weat.lower()
                #s = 0
                #s += (1 if('fog' in weat)  else 0)
                #s += (1 if('rain' in weat) else 0)
                #s += (1 if('snow' in weat) else 0)
                event.append(1)
        return event
    
    def preprocessAndGetData(self):
        df = self.df
        aggrData = self.getAggregate()
        aggrDataSet = pd.DataFrame(aggrData, columns = self.formatCol(df['2009'].columns) )
        dropcolumns = []
        for key in aggrDataSet.keys():
            if 'high' in key or 'low' in key:
                dropcolumns.append(key)
        #dropcolumns = ['Temp high (°C)','Temp low (°C)','Dew Point high (°C)','Dew Point low (°C)','Humidity (%) high','Humidity (%) low','Sea Level Press. (hPa) high','Sea Level Press. (hPa) low','Visibility (km) high','Visibility (km) low','Wind (km/h) low','Wind (km/h) high']
        aggrDataSet = aggrDataSet.drop(dropcolumns,axis=1)

        weatherEvent = aggrDataSet['WeatherEvent']
        #fog,rain,snow = self.getWeatherEventData(weatherEvent)
        event = self.getWeatherEventData(weatherEvent)
        prinSum = aggrDataSet['Precip (mm) sum']
        prinSum = [0 if ele == 'T' else ele for ele in prinSum]
        aggrDataSet['Precip (mm) sum'] = prinSum
        aggrDataSet.dropna(0)

        del aggrDataSet['Day']
        del aggrDataSet['WeatherEvent']

        #aggrDataSet['fog'] = fog
        #aggrDataSet['rain'] = rain
        #aggrDataSet['snow'] = snow
        aggrDataSet['event'] = event
        aggrData = aggrDataSet.values
        aggrData[aggrData == '-'] = 0

        aggrGrp = aggrDataSet.groupby(['Year', 'Month']).groups
        finalGrp = {}
        for date, index in aggrGrp.items():
            
            data = aggrData[index]
            #print(date, data.shape)
            #print(data[:,2:-1])
            avgRows = np.mean(data[:, 2:-2], axis=0)
            #stdRows = np.std(data[:, 2:-2].astype(float), axis=0)
            sumRows = np.sum(data[:,-2:],axis = 0)
            updated = avgRows.tolist() + sumRows.tolist()
            #updated = avgRows.tolist() + stdRows.tolist() + sumRows.tolist()
            finalGrp[date] = updated
            #print(sumRows)
            #break
        return finalGrp

In [141]:
weath = weatherData("WeatherData.xlsx")
weatherData = weath.preprocessAndGetData()
#print(len(weatherData.keys()))


In [127]:
print(len(weatherData.keys()))

96


In [142]:
month = {'Jan' : 1, 'Feb' : 2, 'Mar' : 3, 'Apr': 4, 'May' : 5, 'Jun': 6, 'Jul' : 7, 'Aug': 8, 'Sep':9, 'Oct': 10, 'Nov' : 11, 'Dec' : 12}
try:
    #csv_columns = ['Year', 'Month', 'Temp Mean', 'Dew Point Mean', 'Humidity Mean', 'Sea Level Press Mean', 'Visibility Mean','Wind Mean','Temp Std', 'Dew Point Std', 'Humidity Std', 'Sea Level Press Std', 'Visibility Std','Wind Std', 'Precip sum', 'fog','rain','snow']
    #csv_columns = ['Year', 'Month', 'Temp Mean', 'Dew Point Mean', 'Humidity Mean', 'Sea Level Press Mean', 'Visibility Mean','Wind Mean','Temp Std', 'Dew Point Std', 'Humidity Std', 'Sea Level Press Std', 'Visibility Std','Wind Std', 'Precip sum', 'fog','rain','snow']

    #csv_columns = ['Year', 'Month','Temp Std', 'Dew Point Std', 'Humidity Std', 'Sea Level Press Std', 'Visibility Std','Wind Std', 'Precip sum', 'fog','rain','snow']
    csv_columns = ['Year', 'Month','Temp Std', 'Dew Point Std', 'Humidity Std', 'Sea Level Press Std', 'Visibility Std','Wind Std', 'Precip sum', 'event']

    with open('csv/weatherdata_final.csv', 'w') as f:  # Just use 'w' mode in 3.x
        writer = csv.writer(f, dialect='excel', quoting=csv.QUOTE_NONNUMERIC)
        writer.writerow(csv_columns)
        for key,value in weatherData.items():
            y,m = key
            if(str(m) == 'nan'):
                continue
            row = [y, month[m]]
            row = row + value
            writer.writerow(row)
except IOError:
            print("I/O error({0}): {1}".format(errno, strerror))              

In [136]:
'''
df = pd.read_csv('Train_Kaggle.csv')
dfgrp = df.groupby(['Year','Month']).sum()
dfgrp.to_csv('csv/train_updated.csv')
'''

"\ndf = pd.read_csv('Train_Kaggle.csv')\ndfgrp = df.groupby(['Year','Month']).sum()\ndfgrp.to_csv('csv/train_updated.csv')\n"

In [137]:
df = pd.read_csv('csv/holidayData.csv')
months = df['MonthDate']
months = [int(str(date).split('/')[0]) for date in months]
df['MonthDate'] = months
dfg = df.groupby(['Year', 'MonthDate'])

In [138]:
easterEvent = np.zeros((len(months),1))
christmasEvent = np.zeros((len(months),1))
easterEvent[df['Event'] == 'Easter Sunday'] = 1
christmasEvent[df['Event'] == 'Christmas Day'] = 1

In [118]:
dates = np.transpose(np.vstack((df['Year'], df['MonthDate'])))
events = np.concatenate((easterEvent, christmasEvent), axis = 1)
final = np.concatenate((dates,events),axis= 1)
finaldf = pd.DataFrame(final, columns = ['Year', 'Month', 'Easter', 'Christamas'])
finaldf = finaldf.drop_duplicates()

In [139]:
prevRow = [0,0,0,0]
updated = []
i = 0
#updated = np.zeros((finaldf.shape), dtype = float)
for row in (finaldf.values):
    if(prevRow[0] == row[0] and prevRow[1] == row[1]):
        if((prevRow[2:] == 1).any()):
            continue
        elif((row[2:] == 1).any()):
            updated[i-1] = row
    else:
        updated.append(row)
        i += 1
    prevRow = row
updated = np.array(updated)
updated.shape


(81, 4)

In [119]:
finaldf = pd.DataFrame(updated, columns = ['Year', 'Month', 'Easter', 'Christamas'])
finaldf.to_csv('csv/eventdays.csv', index = False)